In [ ]:
import sys
sys.path.append('../')

In [ ]:
from kraken import *

In [ ]:
exchanges = {'Kraken': {}}

In [ ]:
kraken = exchanges['Kraken']

api_key = api_dict['Kraken']['key']
api_sec = api_dict['Kraken']['sec']

kraken['balance'], kraken['prices'], kraken['account_balances'], kraken['currencies'] = kraken_pull_all(api_key, api_sec)

In [ ]:
# COMBINE DAILY SALES WITH OUR BALANCE POSITIONS
daily_data = kraken['prices'].merge(kraken['balance'], how='outer', left_index=True, right_index=True)
daily_data = daily_data.fillna(0)

for asset in kraken['currencies']:
    daily_data[f'cum_{asset}'] = (daily_data[asset]).cumsum()
    daily_data = daily_data.drop(columns={asset}).rename(columns={f'cum_{asset}':asset})
    
    if asset not in fiat_currencies:
        daily_data[f'{asset}$'] = daily_data[asset]*daily_data[f'{asset}/USD']

In [ ]:
import plotly.graph_objects as go

df = daily_data[daily_data.index >= '2021-01-20'].copy()
    
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df.index, y=df["DOGE$"],
    line_color='rgba(255,255,255,0)',
    fillcolor='rgba(203,152,0,0.5)',
    fill='tonexty',
    name='DOGE',
))
fig.add_trace(go.Scatter(
    x=df.index, y=df["XMR$"]+df["DOGE$"],
    fill='tonexty',
    line_color='rgba(255,255,255,0)',
    fillcolor='rgba(0,100,800,0.5)',
    name='XMR',
))
fig.add_trace(go.Scatter(
    x=df.index, y=df["ADA$"]+df["XMR$"]+df["DOGE$"],
    fill='tonexty',
    line_color='rgba(255,255,255,0)',
    fillcolor='rgba(0,1000,800,0.5)',
    name='ADA',
))
fig.add_trace(go.Scatter(
    x=df.index, y=df["DOT$"]+df["ADA$"]+df["XMR$"]+df["DOGE$"],
    fill='tonexty',
    line_color='rgba(255,255,255,0)',
    fillcolor='rgba(0,100,100,100.5)',
    name='DOT',
))
fig.add_trace(go.Scatter(
    x=df.index, y=df["ETH$"]+df["DOT$"]+df["XMR$"]+df["ADA$"]+df["DOGE$"],
    fill='tonexty',
    line_color='rgba(255,255,255,0)',
    fillcolor='rgba(0,10,8000,0.5)',
    name='ETH',
))
fig.add_trace(go.Scatter(
    x=df.index, y=df["BTC$"]+df["ETH$"]+df["DOT$"]+df["XMR$"]+df["ADA$"]+df["DOGE$"],
    fill='tonexty',
    line_color='rgba(255,255,255,0)',
    fillcolor='rgba(2030,152,0,0.5)',
    name='BTC',
))
fig.add_trace(go.Scatter(
    x=df.index, y=df["USD"]+df["BTC$"]+df["ETH$"]+df["DOT$"]+df["XMR$"]+df["ADA$"]+df["DOGE$"],
    fill='tonexty',
    line_color='rgba(255,255,2550,0)',
    fillcolor='rgba(0,1000,80,0.5)',
    name='USD',
))

fig.update_traces(mode='lines')
fig.show()